## Use Sportradar's NBA API to build your own database

### Introduction

My disovery of advanced statistics in sports is one of the things that led me to data science. Watching sports on TV usually means putting up with a slew of tired observations and adages of questionable accuracy. It was fresh and interesting to learn about concepts like value over replacement and points per possession. No one number can capture a player's impact, but advanced statistics are much more useful than the other information available. 

The NBA is my favorite league and it would be very fun to do data analysis. Before that, I must do the less glamorous task of preparing the data. Which, according to [some estimates](https://www.anaconda.com/state-of-data-science-2020), is 45% of a data scientist's working hours. I identified the [sportradar API](https://developer.sportradar.com/docs/read/basketball/NBA_v7) as an option for data. However, a SQL database would be much more appealing for the data combinations, custom statistics, and other work that would require the data in tabular format. Additionally, we have to consider the cost of requesting data from an API so frequently.

Currently, I have been able to create two tables in my NBA database: the schedules table and the seasons table. In this post, I will show how I:

1. Connected to the sportradar API
2. Downloaded NBA season and schedule data
3. Flattened the data and load it into a relational database
4. Developed procedures for updates that only download what is necessary

Oh, and a final personal note: this is my first Python post! I will be doing more of these moving forward.

### Pre-requisites

1. A [sportradar API](https://developer.sportradar.com/docs/read/basketball/NBA_v7) key
2. A SQL database that you have successfully connected to and can modify. A tutorial to create and connect to an Azure SQL database in Python can be found [here](https://docs.microsoft.com/en-us/azure/azure-sql/database/connect-query-python?view=azuresql). 
3. Python and all packages used in the post
4. (Optional) Some familiarity with environment variables
5. (Optional) An Azure KeyVault account. See how to get set up and interact with Azure KeyVault via Python [here](https://docs.microsoft.com/en-us/azure/key-vault/secrets/quick-create-python?tabs=azure-cli).

### Getting set up

To begin, let's load the packages and specify the folder paths we will be using. Like previous posts, I will be using a JSON configuration file and environment variables to define many of these things. It keeps information private while still letting me share the fun parts. Additionally, there are a few custom functions from my main development project. Functions from that module connect to the database and my key vault account.

The calls to `importlib` and `types` allow me to import the `myFuns` module I have been coding in my main project folder. It is appropriately named: myFuns has all of the functions I have been using to build the database, including connections and secret handling. If you'd prefer not mess with those sorts of things, define the following variables to proceed: 

1. An object `srNBAKey` that is your sportsradar API key for the NBA v7 API
2. An object `cnxn` that is your connection to the SQL database engine
3. An object `nbaDir` that will house the downloaded JSON files

Now, let's run the code to set up our session:

In [1]:
# SETUP
# Import standard packages
import os, importlib.machinery, http.client, json, pandas as pd, time
import sqlalchemy
import importlib, types # Only needed if loading custom module

# Read in Python config file
cfFn = os.path.join(os.environ["myconfig"], "nba.json")
with open(cfFn, "r") as f:
    cf = json.load(f)

# Source custom functions
myFunsFn = os.path.join(cf["nbaFunsDir"], "myFuns/__init__.py")
sys.path.append(cf["nbaFunsDir"])
from myFuns.cloud import secman

# Get NBA API key from Azure and connect to SQL db
seccli = secman.secretClient()
srNBAKey = seccli.get_secret("srNBAKey")
cnxn = secman.dbEngine(seccli)

# Specify directories for output
nbaDir = "D:/nbaBlog"
schedDir = os.path.join(nbaDir, "schedules")
# os.makedirs(schedDir)

Wonderful! We have our API key ready to go and we are connected to our database. Now, let's explore the sportradar API.

### Explore the API

When you navigate to the [API main page](https://developer.sportradar.com/docs/read/basketball/NBA_v7#nba-api-map), you will be greeted with this image:

![sportradar API diagram](https://developer.sportradar.com/files/NBAv7SVG.svg)

Quite helpful! Today, we will be working with the Schedule and Seasons endpoints. Let's take a peek at the data. We will start an https connection, send a get request with our API key, and load the JSON payload.

In [18]:
# Get list of available seasons
conn = http.client.HTTPSConnection("api.sportradar.us", timeout = 10)
suffix = f"/nba/trial/v7/en/league/seasons.json?api_key={srNBAKey.value}"
conn.request("GET", suffix)
res = conn.getresponse()
data = res.read()
allSeasons = json.loads(data)['seasons']
print(json.dumps(allSeasons[0:2], indent = 2))

[
  {
    "id": "5b8b57d1-7290-44f8-b29c-353e865c139e",
    "year": 2012,
    "type": {
      "code": "PRE",
      "name": "Pre-season"
    }
  },
  {
    "id": "5f45c666-ba68-48d6-a905-f19702ab7e4c",
    "year": 2012,
    "type": {
      "code": "PST",
      "name": "Post-season"
    }
  }
]


The schedule has games for the pre-season, post-season, and regular season. Each season has a unique ID and year. Next, let's pull one of these seasons and look at its schedule. We will need the year and type of a season to pull it's schedule.

In [16]:
# Get year and season type for first listed
year = allSeasons[1]["year"]
seasonType = allSeasons[1]["type"]["code"]

# Pull the data, convert to JSON object
# Add some rests to prevent errors
conn = http.client.HTTPSConnection("api.sportradar.us", timeout = 10)
conn.request("GET", f"/nba/trial/v7/en/games/{year}/{seasonType}/schedule.json?api_key={srNBAKey.value}")
res = conn.getresponse()
data = res.read()
testSched = json.loads(data.decode("utf8"))

[]


### Design the database tables

Now that we have our two file formats, we can start to think through a database design. Reading through the docs, it seems the table design is not [normalized](https://docs.microsoft.com/en-us/office/troubleshoot/access/database-normalization-description). For example, there is a team profile API endpoint containing most of the team information included in the `home` and `away` entries, so the same information is being presented multiple times. This makes sense: it likely reduces the number of API calls for most users. Still, this underscores the point that the optimal database design is not the same as the optimal API design. We are not working with a particularly sizable dataset, so it should be easy to rebuild the database if we want to change the design later. 

After some initial experimentation, I decided that these were the variables I wanted:


In [4]:
cnxn.execute("""
DROP TABLE IF EXISTS scheduleBLOG;
CREATE TABLE scheduleBLOG (
    schedId varchar(100) PRIMARY KEY        
    , srMatchId varchar(100)
    , schedDt DATETIME
    , homePoints int
    , awayPoints int
    , coverage varchar(50)
    , status varchar(50)    
    , trackOnCourt varchar(20)
    , IdHome varchar(100)
    , srIdHome varchar(100)
    , nameHome varchar(100)
    , aliasHome varchar(100)
    , IdAway varchar(100)
    , srIdAway varchar(100)
    , nameAway varchar(100)
    , aliasAway varchar(100)
    , seasonId VARCHAR(100) NOT NULL 
    , createDt DATETIME DEFAULT CURRENT_TIMESTAMP
    , updatedDt DATETIME     
);

DROP TABLE IF EXISTS seasonBLOG;
CREATE TABLE seasonBLOG (
    seasonId VARCHAR(100) NOT NULL PRIMARY KEY
    , seasonType varchar(10) NOT NULL
    , seasonYear int NOT NULL
    , createDt DATETIME DEFAULT CURRENT_TIMESTAMP
    , updatedDt DATETIME
);

ALTER TABLE scheduleBLOG
ADD CONSTRAINT FK_seasonIdBLOG
FOREIGN KEY (seasonId) REFERENCES seasonBLOG(seasonId)
""")


We will use the sportradar identifiers as our primary keys in the database for now. We will allow some missing data in the schedule table. After we ingest all of the data, we can think more thoroughly about other constraints to add.

The [foreign key](https://www.techopedia.com/definition/7272/foreign-key#:~:text=A%20foreign%20key%20is%20a,establishing%20a%20link%20between%20them.) constraint lets our database know that the schedule and season tables share a common identifier, `seasonId`. That adds some built-in data quality checks. For example, it would not allow you to enter a `seasonId` in the `schedules` table unless that ID were in the `season` table. Formalizing these sorts of relationships help others quickly understand how your database is designed.

### Download Data

Now that we have explored the data and created our database tables, we will write code to download all data in JSON format and save it in an organized fashion. All files will be named by the convention `schedTYPEYYYY.json` (e.g. `schedPRE2022`). We also want to avoid re-downloading data and making unnecessary API calls. The `os.path.exists` function comes in quite handy. I have already downloaded most of the data, but for the sake of this blog I will delete a few schedules and run the function. 

In [6]:
# Turn a schedule retrieval into a fnxn
def getSchedule(year, seasonType, apiKey):
    conn = http.client.HTTPSConnection("api.sportradar.us", timeout = 10)    
    conn.request("GET", f"/nba/trial/v7/en/games/{year}/{seasonType}/schedule.json?api_key={apiKey}")
    res = conn.getresponse()
    data = res.read()    
    return json.loads(data.decode("utf8"))

# Download each season if its key is not yet in the database
# Template names based on season type and year
for season in allSeasons:         
    year = season["year"]
    type = season["type"]["code"]
    fileName = "sched" + type + str(year) + ".json"
    outpath = os.path.join(schedDir, fileName)

    # Only download file if we have not already    
    if not os.path.exists(outpath):
        print("downloading "+ fileName)
        time.sleep(1)
        seas = getSchedule(year = year, seasonType = type, apiKey = srNBAKey.value)
        with open(outpath, 'w') as f:               
            json.dump(seas, f)

downloading schedPST2014.json
downloading schedPST2015.json
downloading schedPST2016.json
downloading schedPRE2022.json


### Upload Data

Now, we put the data we just downloaded into our database. It will be easiest to rename the JSON data elements to match their SQL-database counterparts. To do that, we use our trustry old friend, the JSON file. The dictionary keys will be the SR API names, and the values our desired values. The JSON file looks like this:

In [7]:
# JSON file with schedionaries for renaming raw to SQL column names
with open("renameVars.json", "r") as f:
    renames = json.load(f)

print(json.dumps(renames, indent = 2))

{
  "schedule": {
    "id": "schedId",
    "sr_id": "srMatchId",
    "scheduled": "schedDt",
    "home_points": "homePoints",
    "away_points": "awayPoints",
    "coverage": "coverage",
    "status": "status",
    "track_on_court": "trackOnCourt",
    "idHome": "idHome",
    "sr_idHome": "srIdHome",
    "nameHome": "nameHome",
    "aliasHome": "aliasHome",
    "idAway": "idAway",
    "sr_idAway": "srIdAway",
    "nameAway": "nameAway",
    "aliasAway": "aliasAway"
  },
  "season": {
    "id": "seasonId",
    "type": "seasonType",
    "year": "seasonYear"
  }
}


And with that, we should have everything we need to populate our database tables! For both games and seasons, we will want to make sure the record has not been entered before. It also seems there are some seasons with no data available, so the case where the JSON file is empty will need to be handled.

In [9]:
games = pd.read_sql("SELECT DISTINCT schedId FROM scheduleBLOG", cnxn)
seasons = pd.read_sql("SELECT DISTINCT seasonId FROM seasonBLOG", cnxn)

####################################################
# MAKE ROWS
####################################################
# Get all of our schedule JSON files
fns = os.listdir(schedDir)

# Loop through each schedule year and add to db
for fn in fns:     
    # Open file, determine year
    with open(os.path.join(schedDir, fn)) as f:
        sched = json.load(f)
    seas = {renames["season"].get(k, k): v for k, v in sched['season'].items()}    
    season = pd.DataFrame(seas, index = [0])

    # Have we loaded this season before?
    newseason = not season['seasonId'].values[0] in seasons['seasonId'].tolist()
    if newseason:
        print("Adding Season " + fn)
        season.to_sql('seasonBLOG', con = cnxn, if_exists = 'append', index = False)

    # If no games, skip
    if len(sched['games']) == 0:
        print("No games in list " + fn)
        pass

    # Loop through games, extract values into dictionary
    out = []    
    for game in sched['games']:
        newgame = not game['id'] in games['schedId'].tolist()
        if newgame:
            # Variables of interest from parent values
            parentKeep = {'id', 'sr_id', 'status', 'coverage', 'scheduled', \
                'track_on_court', 'home_points', 'away_points'}
            parentValues = {x: game[x] for x in parentKeep if x in game}

            # We want the same values from "home" and "away" subdicts
            teamKeep = {"id", "sr_id", "name", "alias"}
            homeValues = {x + 'Home': game['home'][x] for x in teamKeep \
                if x in game['home']}
            awayValues = {x + 'Away': game['away'][x] for x in teamKeep \
                if x in game['away']}

            # Combine all dicts, rename according to renaming dictionary
            # Use None-type if variable is not present
            comb = {**parentValues, **homeValues, **awayValues}
            res = {renames["schedule"].get(k, k): v for k, v in comb.items()}
            notIn = [x for x in renames['schedule'].values() if x not in res]
            for n in notIn:
                res[n] = None
            out.append(res)
        else: 
            pass

    # Convert all of our dicts to a DF, then load to DB
    print("loading " + fn)
    toLoad = pd.DataFrame.from_dict(out)
    if not toLoad.empty:
        toLoad['schedDt'] = pd.to_datetime(toLoad['schedDt'])
        toLoad['seasonId'] = season['seasonId'].values[0]        
        toLoad.to_sql('scheduleBLOG', con = cnxn, if_exists = 'append', index = False, \
            dtype = {"schedDt": sqlalchemy.DateTime})
    else:
        print("skipping " + fn + "due to no new records")

loading schedPRE2012.json
skipping schedPRE2012.jsondue to no new records
loading schedREG2012.json
skipping schedREG2012.jsondue to no new records
loading schedREG2013.json
Adding Season schedPRE2014.json
loading schedPRE2014.json
Adding Season schedREG2014.json
loading schedREG2014.json
Adding Season schedPRE2015.json
loading schedPRE2015.json
Adding Season schedREG2015.json
loading schedREG2015.json
Adding Season schedPRE2016.json
loading schedPRE2016.json
Adding Season schedREG2016.json
loading schedREG2016.json
Adding Season schedPRE2017.json
loading schedPRE2017.json
Adding Season schedPST2017.json
loading schedPST2017.json
Adding Season schedREG2017.json
loading schedREG2017.json
Adding Season schedPST2018.json
loading schedPST2018.json
Adding Season schedREG2018.json
loading schedREG2018.json
Adding Season schedPIT2019.json
loading schedPIT2019.json
Adding Season schedREG2019.json
loading schedREG2019.json
Adding Season schedPRE2020.json
loading schedPRE2020.json
Adding Season 

Let's make sure that worked. We will query the tables we loaded to and print out a few rows. Did it work? Did it fail? The suspense! The intrigue!

In [11]:
# Read in our newly loaded data
games = pd.read_sql("SELECT TOP 10 * FROM scheduleBLOG", cnxn)
seasons = pd.read_sql("SELECT TOP 10 * FROM seasonBLOG", cnxn)

# Print a few games out
print(games)
# And a few seasons
print(seasons)

                                schedId          srMatchId  \
0  000193f7-3433-461a-b562-7f7c69e8023f               None   
1  000477ca-9053-4bdf-857b-629ebc8e670e  sr:match:24750712   
2  00096a31-9afb-4d89-957f-2ca5741f813c  sr:match:12233334   
3  000bee26-0d7f-451e-b2c3-b52f24581c7f   sr:match:7790366   
4  000e7d9f-7fa0-4273-a419-e02a95cd6101   sr:match:9956289   
5  0010abb3-bf15-401a-97fd-a20d7f45ef3b               None   
6  00189aa2-fe36-420e-b3a1-7d9dbd8846a1  sr:match:28808730   
7  001b86bd-1891-462f-8d0b-c373434d4f14  sr:match:15327444   
8  001da480-f3ad-4da4-a72b-9899f2394672  sr:match:28809934   
9  001eaa7b-4a2d-4077-b590-1f288a42a8a2   sr:match:4194739   

              schedDt  homePoints  awayPoints coverage     status  \
0 2023-04-08 00:30:00         NaN         NaN     full  scheduled   
1 2021-02-27 03:00:00       130.0       121.0     full     closed   
2 2017-12-14 00:00:00        95.0       106.0     full     closed   
3 2015-10-31 00:00:00       113.0       1

Looks to have worked. Eventually, a custom primary key that sorts chronologically would be nice. But, we have a good prototype to work with. We can now download some game/season statistic data and start having some real fun in the coming weeks and months. At least it will give me something to do this NBA season while I nervously wait for Chet Holmgren to [heal from his severe foot injury](https://theathletic.com/3535418/2022/08/24/thunder-chet-holmgren-out-for-season-lisfranc-injury/). Feel better soon, Chet!